In [1]:
from urllib.request import urlopen
import os
from bs4 import BeautifulSoup as BeautifulSoup
import pandas as pd

In [36]:
# Step 1: Define funtions to download filings
def get_list(ticker, file_type, todate):

    base_url_part1 = "http://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK="
    base_url_part2 = "&type=&dateb=&owner=&start="
    base_url_part3 = "&count=100&output=xml"
    href = []
    
    for page_number in range(0,2000,100):
    
        base_url = base_url_part1 + ticker + base_url_part2 + str(page_number) + base_url_part3
        
        sec_page = urlopen(base_url)
        sec_soup = BeautifulSoup(sec_page)
        
        filings = sec_soup.findAll('filing')
        
        for filing in filings:
            report_year = int(filing.datefiled.get_text()[0:4])
            if (filing.type.get_text() == file_type) & (report_year > todate):
                print (filing.filinghref.get_text())
                href.append(filing.filinghref.get_text())
    
    return href

def download_report(url_list,dir_path, typefile):
    
    target_base_url = 'http://www.sec.gov'
    
    # type = 'EX-101.INS'
    target_file_type = typefile
    
    for report_url in url_list:
        report_page = urlopen(report_url)
        report_soup = BeautifulSoup(report_page)
        
        xbrl_file = report_soup.findAll('tr')
        #print (xbrl_file)
        for item in xbrl_file:
            try:
                if item.findAll('td')[3].get_text() == target_file_type:
                    if not os.path.exists(dir_path):
                        os.makedirs(dir_path)
                    
                    end_path=item.findAll('td')[2].find('a')['href']
                    end_path=end_path.split('/')
                    #print (end_path[1])
                    if end_path[1]=='ix?doc=':
                        del end_path[1]
                    end_path=','.join(end_path)
                    end_path=end_path.replace(',', '/')
                    #print (end_path)
                    target_url = target_base_url + end_path
                    print ("Target URL found!")
                    print ("Target URL is:", target_url)
                    
                    file_name = target_url.split('/')[-1]
                    print (file_name)
                   
                    xbrl_report = urlopen(target_url)
                    output = open(os.path.join(dir_path,file_name),'wb')
                    output.write(xbrl_report.read())
                    output.close()
                    
            except:
                pass

In [45]:
# Step 2: Define funtions to download filings
# Import tickers
# TickerFile = pd.read_csv("companylist.csv")
# Tickers = TickerFile['Symbol'].tolist()

# tickers = Tickers

# for ticker in tickers:
def getfiles(tickername, filename, year):   
    ticker=tickername
    url_list= get_list(ticker, filename,year-1)
    base_path = "C:/data/CS/project3"
    dir_path = base_path + "/"+ticker+"/"+filename
    download_report(url_list,dir_path, filename)

In [46]:
getfiles ('ETSY','8-K', 2019)
getfiles('ETSY', '10-Q', 2019)

https://www.sec.gov/Archives/edgar/data/1370637/000119312519252365/0001193125-19-252365-index.htm
https://www.sec.gov/Archives/edgar/data/1370637/000119312519248215/0001193125-19-248215-index.htm
https://www.sec.gov/Archives/edgar/data/1370637/000137063719000157/0001370637-19-000157-index.htm
https://www.sec.gov/Archives/edgar/data/1370637/000137063719000143/0001370637-19-000143-index.htm
https://www.sec.gov/Archives/edgar/data/1370637/000119312519198611/0001193125-19-198611-index.htm
https://www.sec.gov/Archives/edgar/data/1370637/000137063719000107/0001370637-19-000107-index.htm
https://www.sec.gov/Archives/edgar/data/1370637/000137063719000087/0001370637-19-000087-index.htm
https://www.sec.gov/Archives/edgar/data/1370637/000137063719000017/0001370637-19-000017-index.htm
Target URL found!
Target URL is: http://www.sec.gov/Archives/edgar/data/1370637/000119312519252365/d672648d8k.htm
d672648d8k.htm
Target URL found!
Target URL is: http://www.sec.gov/Archives/edgar/data/1370637/0001193